# Modified Sigmoid

Sigmoid is apparently [super bad](https://krbnite.github.io/ReLU-vs-Sigmoid-vs-Tanh/) for deep learning. 

Can we make them suck less?

In [3]:
from fastai.basics import *
from fastai.vision.all import *

In [4]:
path = untar_data(URLs.IMAGENETTE)

A new version of this dataset is available, downloading...


File downloaded is broken. Remove /home/josh/.fastai/archive/imagenette2.tgz and try again.


## Baseline

In [5]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=32)

In [8]:
model = xresnet50(n_out=dls.c)

In [11]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.638726,2.427304,0.396191,01:18
1,1.232394,1.203652,0.620612,01:25
2,0.949838,0.987503,0.702390,01:23
3,0.701514,0.623808,0.803958,01:22
4,0.604172,0.547519,0.825990,01:18


## Modified Sigmoid

We want a sigmoid but we want to lie about the gradient.

In [6]:
class ModifiedSigmoid(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        return torch.nn.functional.sigmoid(input)
    
    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return grad_input

In [15]:
class ModifiedSigmoidModule(nn.Module):
    
    def __init__(self):
        super(ModifiedSigmoidModule, self).__init__()
        
    def foward(self, x):
        pass
        # TODO: Call ModifiedSigmoid somehow?
        # Just need to call ModifiedSigmoid.apply() or something. See:
        # https://github.com/pytorch/pytorch/blob/master/torch/autograd/function.py#L141-L155
        
    

Now we apply it to each Relu

In [67]:
def convert_relu_to_softplus(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            if str.isdigit(child_name):
                print("Model", model)
                print("child_name", child_name)
                model[int(child_name)] = torch.nn.functional.sigmoid()
            else:
                setattr(model, child_name, torch.nn.functional.sigmoid())
        else:
            convert_relu_to_softplus(child)